In [ ]:
import sys
import os
os.chdir('/home/usuario/Documents/Barcelona_Yr1/GraphicalModels_NetworkData/LiLicode/paper_code_github/')
sys.path.append("/Network_Spike_and_Slab/numpyro/functions")

sim_data_path = './Data/Simulations/'
data_save_path = './data/'

In [ ]:
import models
import my_utils

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from IPython.display import display
from IPython.display import display_html 

In [ ]:
import jax
import numpyro
numpyro.set_platform('cpu')
print(jax.lib.xla_bridge.get_backend().platform)
import jax.numpy as jnp

In [ ]:
n_sims = 2
ps = [10, 50]
n = 2000
n_cuts = [100, 200, 500]

# Compile results

In [ ]:
outputs_2mcmc_all = {p:{} for p in ps}
for p_ix, p in enumerate(ps):
    for n_cut in n_cuts:
        # 2nd MCMC
        output_dict_glasso_ss = {"TP":[], "FP":[], "TN":[], "FN":[], "TPR":[], "FPR":[], 
                                 "FNR":[], "FDiscR":[], "FNonDiscR":[],
                       "TP_95":[], "FP_95":[], "TN_95":[], "FN_95":[], 
                                 "TPR_95":[], "FPR_95":[], "FNR_95":[], 
                                 "FDiscR_95":[], "FNonDiscR_95":[],
                       "mse_theta":[]}

        output_dict_ss_85SEMI = {"mse_theta":[], "TP":[], "FP":[], "TN":[], "FN":[], "TPR":[], "FPR":[], 
                               "FNR":[], "FDiscR":[], "FNonDiscR":[],
                                "TP_95":[], "FP_95":[], "TN_95":[], "FN_95":[],
                               "TPR_95":[], "FPR_95":[], "FNR_95":[], "FDiscR_95":[], "FNonDiscR_95":[],}

        output_dict_ss_SEMI = {"mse_theta":[], "TP":[], "FP":[], "TN":[], "FN":[], "TPR":[], "FPR":[], 
                                  "FNR":[], "FDiscR":[], "FNonDiscR":[], 
                                "TP_95":[], "FP_95":[], "TN_95":[], "FN_95":[],
                                  "TPR_95":[], "FPR_95":[], "FNR_95":[], 
                                 "FDiscR_95":[], "FNonDiscR_95":[]}

        output_dict_ss_INDEP = {"mse_theta":[], "TP":[], "FP":[], "TN":[], "FN":[], "TPR":[], "FPR":[], 
                                "FNR":[], "FDiscR":[], "FNonDiscR":[], 
                                "TP_95":[], "FP_95":[], "TN_95":[], "FN_95":[],
                                "TPR_95":[], "FPR_95":[], "FNR_95":[], 
                               "FDiscR_95":[], "FNonDiscR_95":[]}

        outputs = { "SS":output_dict_glasso_ss,
                      "NetworkSS_85SEMI":output_dict_ss_85SEMI, "NetworkSS_SEMI":output_dict_ss_SEMI, 
                    "NetworkSS_INDEP":output_dict_ss_INDEP}

        for s in range(n_sims):

            with open(data_save_path + f'sim_SS_data/SS_{s}_p{p}_n{n_cut}_2mcmc.sav', 'rb') as fr:
                res_glasso_ss = pickle.load(fr)

            with open(data_save_path + f'sim_NetworkSS_data/NetworkSS_A85semi_{s}_p{p}_n{n_cut}_2mcmc.sav', 'rb') as fr:
                res_ss_85SEMI = pickle.load(fr)
            with open(data_save_path + f'sim_NetworkSS_data/NetworkSS_A75semi_{s}_p{p}_n{n_cut}_2mcmc.sav', 'rb') as fr:
                res_ss_SEMI = pickle.load(fr)
            with open(data_save_path + f'sim_NetworkSS_data/NetworkSS_Aindep_{s}_p{p}_n{n_cut}_2mcmc.sav', 'rb') as fr:
                res_ss_INDEP = pickle.load(fr)


            with open(sim_data_path + f'sim{s}_p{p}_n{n}.sav', 'rb') as fr:
                sim_res = pickle.load(fr)

            all_res = {"SS":res_glasso_ss,
                      "NetworkSS_85SEMI":res_ss_85SEMI, "NetworkSS_SEMI":res_ss_SEMI, "NetworkSS_INDEP":res_ss_INDEP}
            for i, res in all_res.items():


                try:
                    TP = res['TP']
                    FP = res['FP']
                    TN = res['TN']
                    FN = res['FN']

                    outputs[i]["TP"].append(TP)
                    outputs[i]["FP"].append(FP)
                    outputs[i]["TN"].append(TN)
                    outputs[i]["FN"].append(FN)

                    outputs[i]["TPR"].append(my_utils.get_TPR(TP,FN))
                    outputs[i]["FPR"].append(my_utils.get_FPR(FP, TN))
                    outputs[i]["FNR"].append(my_utils.get_FNR(FN=FN, TP=TP))
                    try:
                        outputs[i]["FDiscR"].append(my_utils.get_FDiscR(FP=FP, TP=TP))
                    except:
                        outputs[i]["FDiscR"].append(0)
                    try:
                        outputs[i]["FNonDiscR"].append(my_utils.get_FNonDiscR(TN=TN, FN=FN))
                    except:
                        outputs[i]["FNonDiscR"].append(0)

                except:

                    TP_5 = res['TP_5']
                    FP_5 = res['FP_5']
                    TN_5 = res['TN_5']
                    FN_5 = res['FN_5']

                    outputs[i]["TP"].append(TP_5)
                    outputs[i]["FP"].append(FP_5)
                    outputs[i]["TN"].append(TN_5)
                    outputs[i]["FN"].append(FN_5)

                    outputs[i]["TPR"].append(my_utils.get_TPR(TP=TP_5, FN=FN_5))
                    outputs[i]["FPR"].append(my_utils.get_FPR(FP=FP_5, TN=TN_5))
                    outputs[i]["FNR"].append(my_utils.get_FNR(FN=FN_5, TP=TP_5))
                    try:
                        outputs[i]["FDiscR"].append(my_utils.get_FDiscR(FP=FP_5, TP=TP_5))
                    except:
                        outputs[i]["FDiscR"].append(0)
                    try:
                        outputs[i]["FNonDiscR"].append(my_utils.get_FNonDiscR(TN=TN_5, FN=FN_5))
                    except:
                        outputs[i]["FNonDiscR"].append(0)

                    TP_95 = res['TP_95']
                    FP_95 = res['FP_95']
                    TN_95 = res['TN_95']
                    FN_95 = res['FN_95']

                    outputs[i]["TP_95"].append(TP_95)
                    outputs[i]["FP_95"].append(FP_95)
                    outputs[i]["TN_95"].append(TN_95)
                    outputs[i]["FN_95"].append(FN_95)

                    outputs[i]["TPR_95"].append(my_utils.get_TPR(TP=TP_95, FN=FN_95))
                    outputs[i]["FPR_95"].append(my_utils.get_FPR(FP=FP_95, TN=TN_95))
                    outputs[i]["FNR_95"].append(my_utils.get_FNR(FN=FN_95, TP=TP_95))
                    try:
                        outputs[i]["FDiscR_95"].append(my_utils.get_FDiscR(FP=FP_95, TP=TP_95))
                    except:
                        outputs[i]["FDiscR_95"].append(0)
                    try:
                        outputs[i]["FNonDiscR_95"].append(my_utils.get_FNonDiscR(TN=TN_95, FN=FN_95))
                    except:
                        outputs[i]["FNonDiscR_95"].append(0)        

                outputs[i]["mse_theta"].append(my_utils.get_MSE(res['all_samples']["theta"].mean(0),
                                                                sim_res["theta_true"]))

        outputs_2mcmc = outputs
        outputs_2mcmc_all[p][n_cut] = outputs_2mcmc
        

# Summary for specific p, n_cut

In [ ]:
p = 10
n_cut = 100

outputs_2mcmc = outputs_2mcmc_all[p][n_cut]

In [ ]:
cols = ['mse_theta', 'TP_95', 'TN_95', 'FP_95', 'FN_95', 
        "TPR_95","FPR_95", "FNR_95", 
         "FDiscR_95", "FNonDiscR_95"]

res_dict_2mcmc = {"SS":[],
              "NetworkSS_85SEMI":[], "NetworkSS_SEMI":[], "NetworkSS_INDEP":[]}


for k, output in outputs_2mcmc.items():
    for col in cols:
        try:
            res_dict_2mcmc[k].append(round(jnp.array(output[col]).mean(0),3))
        except:
            res_dict_2mcmc[k].append(np.nan)

### *Results for 2 MCMC*

In [ ]:
pd.DataFrame.from_dict(res_dict_2mcmc, orient='index', columns=cols).loc[['SS', 'NetworkSS_85SEMI',
                                                                        'NetworkSS_SEMI', 'NetworkSS_INDEP']]